### RAG with PDF 📄 Data extraction to give context to LLM 🧠

<img src="./Images/RAGs.png" width="800" height="700" style="display: block; margin: auto;">

In [1]:
!pip install pypdf


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
load = load_dotenv('./../.env')


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen3:1.7b",
    temperature=0.5,
    max_tokens = 250
)

### 1. Extracting the PDF files

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf1 = "./attention.pdf"
pdf2 = "./LLMForgetting.pdf"
pdf3 = "./TestingAndEvaluatingLLM.pdf"

pdfFiles = [pdf1, pdf2, pdf3]

documents = []

for pdf in pdfFiles:
    loader = PyPDFLoader(pdf)
    documents.extend(loader.load())

print(len(documents))

In [ ]:
print(documents[:1])

### 2. Text Splitting into Chunks

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap=200, add_start_index=True)

all_splits = text_splitter.split_documents(documents)

len(all_splits)

### 3. Embedding

In [ ]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2:latest")

vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
len(vector_1), len(vector_1)

### 4. Vector Stores

In [ ]:
#!pip install -qU langchain-chroma

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents = all_splits,
    embedding=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

### 5. Retriving from the Persistant Vector Datastore

In [ ]:
from langchain_chroma import Chroma


vector_store = Chroma(persist_directory='./chroma_langchain_db', embedding_function=embeddings)

result = vector_store.similarity_search("What is Bias testing", k=3)

for doc in result:
    print(doc.page_content)

In [ ]:
result = vector_store.similarity_search_with_score("What are the types of LLM Testing")

result[0]

### 6. Retrivers in Langchain

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs = {"k": 1}
)

retriever.batch(
    [
        "What is the Bias Measurement",
        "How to test human safety against LLM",
        "How LLM forgets the context"
    ]
)


### Document Retrival Manually

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

query = "What exactly does Testing the Factual Correctness of LLM tells"

retrieved_docs = retriever.get_relevant_documents(query)

context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt_template = ChatPromptTemplate.from_template(
    """
    You are an AI Assisant. Use the following context to answer the question correctly.
    If you dont know the answer, just tell, I dont know.
    
    Also, summarize the response in MD format
    
    "context: {context} \n\n"
    "question: {question} \n\n"
    "AI answer:
    
    """
)

chain = prompt_template | llm | StrOutputParser()

response = chain.invoke({"context": context_text, "question": query})

print(response)

### Using Langchain Hub for Prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

query = "How to test Translation in LLM?"

retrieved_docs = retriever.get_relevant_documents(query)

context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

prompt = hub.pull("rlm/rag-prompt")

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"context": context_text, "question": query})

print(response)

### Retrieving data using RetrievalQA

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever, return_source_documents=True)

question = "What exactly does Testing the Factual Correctness of LLM tells"

response = qa_chain.invoke(question)

sources = set(doc.metadata.get("source", "Unknown") for doc in response["source_documents"])

print(response['result'])
print("\n📕 Sources Used:")
for source in sources:
    print(f"- {source}")